In [1]:
# 
# Module Zeno
# RLH 1/29/2021. updated 11/26/21. Added Lt, step transmission length 12/4/21.
#
module zeno

@doc """
    zenogrowth(Lmax, Hmax, tmax; F=0.001, a=1.0, Ls=1.0, Lc=2.0, Lt=0.0, Ld=1.0, B=0.0)

Minimal model for growth in 1+1 dimensions.

Example: ```zenogrowth(200,400,200000)```
Create a mound with a fixed base layer 200 units long, with a maximum height of 400, calculated in 200k time steps. 
Return Hbot, Htop, L_array, TerraceLength, v.
"""
function zenogrowth(Lmax, Hmax, tmax; F=0.001, a=1.0, Ls=1.0, Lc=2.0, Ld=1.0, B=0.0)   
    
    Hbot = 1
    Htop = 2
    sticking(L) = 1.0-B*((L^2)/(L+Ld)^2) # L-dependent sticking coefficient
    f(L,Ls,isTop) = (isTop ? (a*F*L)*sticking(L) : (a*F*L/2)*sticking(L) * (1 - Ls/(Ls+L))) # from upper terrace
    g(L,Ls,isBot) = (isBot ? (a*F*L)*sticking(L) : (a*F*L/2)*sticking(L) * (1 + Ls/(Ls+L))) # from lower terrace

    v = zeros(Hmax)
    L_array = zeros(Hmax)
    TerraceLength = zeros(Hmax)
    L_array[1] = Lmax


    for i in 1:tmax
        L_0 = copy(L_array)

        # Calculate the terrace lengths.
        TerraceLength = zeros(Hmax)
        for j in Hbot:Htop
            TerraceLength[j] = L_array[j] - L_array[j+1]
        end

        newHbot = 1
        for j in Hbot:Htop
            if L_array[j] >= Lmax
                newHbot = j
            end
        end
        Hbot = newHbot
    
        Htop_temp = copy(Htop)
        Hbot_temp = copy(Hbot)    
    
        # Calculate the step velocities.
        # Changed Htop to Htop-1 to avoid nucleating early. RLH 11/25/21
        v = zeros(Hmax)
        for j in Hbot:Htop-1
            if L_array[j] >= Lmax      
                if L_array[j+1] < Lc
                    v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)       
                else
                    v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                end
                
            elseif L_array[j+1] < Lc
                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
            else
                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
            end
        end
    
        trigger_Lmax = 0 # trigger for Lmax correction
       
        for j in Hbot:Htop
            if L_array[j] >= Lc
                L_array[j+1] += v[j]
                    if L_array[j+1] >= Lmax                   
                        trigger_Lmax = 1 # Lmax correction happened
                    end
            end
        end
    
        trigger_Lc = 0 # trigger for Lc correction
    
        if L_array[Htop] > Lc
            trigger_Lc = 1 # Lc correction needed
        end
        
        # update Htop
        for j in Hbot:Htop
            if L_array[j] >= Lc
                if j==Htop 
                    Htop+=1            
                end
            end
        end
        
        # only Lmax correction is needed, Lc correction is not 
        if trigger_Lmax == 1
            if trigger_Lc == 0
        
                L_step_1 = copy(L_0)  # return to previous step positiion
        
                # return to the old Hot and Hbot
                Htop = copy(Htop_temp)
                Hbot = copy(Hbot_temp)
        
                # Calculate the terrace lengths.
                TerraceLength = zeros(Hmax)
                for j in Hbot:Htop
                    TerraceLength[j] = L_step_1[j] - L_step_1[j+1]
                end
                
                # find length needed to reach exact Lmax
                t_length = TerraceLength[Hbot]

                #flux for sub step 1
                flux_1 =  t_length/(TerraceLength[Hbot]*sticking(TerraceLength[Hbot]) + TerraceLength[Hbot+1]/2*sticking(TerraceLength[Hbot+1])*(1-Ls/(Ls+TerraceLength[Hbot+1])))
   
        
                F_temp = copy(F) # save the default flux
                F = copy(flux_1) # replace the flux with flux_1 for sub step 1

        
                # recalculate the v_array for sub step 1
                v = zeros(Hmax)
                for j in Hbot:Htop-1
                    if L_step_1[j] >= Lmax      
                        if L_step_1[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                        end
            
                    elseif L_step_1[j+1] < Lc
                        v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                    else
                        v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                    end
                end
                            
                #update the L array with sub step 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lc
                        L_step_1[j+1] += v[j]
                    end
                end
            
                # find correct Htop after sub step 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lc
                        if j==Htop 
                            Htop+=1            
                        end
                    end
                end
        
        
                # find correct Hbot for sub step 2
                newHbot = 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lmax
                        newHbot = j
                    end
                end
                Hbot = newHbot
        
                L_step_2 = copy(L_step_1)
        
                flux_2 = F_temp - flux_1
        
                # Calculate the terrace lengths again for sub step 2.
                TerraceLength = zeros(Hmax)
                for j in Hbot:Htop
                    TerraceLength[j] = L_step_2[j] - L_step_2[j+1]
                end
                    
                F = copy(flux_2) # replace the flux with flux_2 for sub step 2
        
                # recalculate the v_array for sub step 2
                v = zeros(Hmax)
                for j in Hbot:Htop-1
                    if L_step_2[j] >= Lmax      
                        if L_step_2[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                        end
            
                    elseif L_step_2[j+1] < Lc
                        v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                    else
                        v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                    end
                end
                
                #update the L array with sub step 2
                for j in Hbot:Htop
                    if L_step_2[j] >= Lc
                        L_step_2[j+1] += v[j]
                    end
                end
            
                # find correct Htop after sub step 2
                for j in Hbot:Htop
                    if L_step_2[j] >= Lc
                        if j==Htop 
                            Htop+=1            
                        end
                    end
                end

                  
                F = copy(F_temp) # return the flux to the default
                L_array = copy(L_step_2)
            end
        end
    
        # only Lc correction is needed, Lmax correction is not 
        if trigger_Lc == 1
            if trigger_Lmax == 0
            
                L_step_1 = copy(L_0)  # return to previous step positiion
        
                # return to the old Hot and Hbot
                Htop = copy(Htop_temp)
                Hbot = copy(Hbot_temp)
        
                # Calculate the terrace lengths.
                TerraceLength = zeros(Hmax)
                for j in Hbot:Htop
                    TerraceLength[j] = L_step_1[j] - L_step_1[j+1]
                end
                
                # find length needed to reach exact Lc
                t_length = Lc - TerraceLength[Htop]

            
                #flux for sub step 1
                flux_1 =  t_length/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]/2*sticking(TerraceLength[Htop-1])*(1+Ls/(Ls+TerraceLength[Htop-1])))
                # special case for layer 1 and 2
                if Hbot == 1
                    if Htop == 2
                        flux_1 = t_length/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]*sticking(TerraceLength[Htop-1]))
                    end
                end
        
                F_temp = copy(F) # save the default flux
                F = copy(flux_1) # replace the flux with flux_1 for sub step 1

        
                # recalculate the v_array for sub step 1
                v = zeros(Hmax)
                for j in Hbot:Htop-1
                    if L_step_1[j] >= Lmax      
                        if L_step_1[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                        end
            
                    elseif L_step_1[j+1] < Lc
                        v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                    else
                        v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                    end
                end
            
                    
                #update the L array with sub step 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lc
                        L_step_1[j+1] += v[j]
                    end
                end
                        
                # find correct Htop after sub step 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lc
                        if j==Htop 
                            Htop+=1            
                        end
                    end
                end
        
        
                # find correct Hbot for sub step 2
                newHbot = 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lmax
                        newHbot = j
                    end
                end
                Hbot = newHbot
        
                L_step_2 = copy(L_step_1)
                flux_2 = F_temp - flux_1
        
                # Calculate the terrace lengths again for sub step 2.
                TerraceLength = zeros(Hmax)
                for j in Hbot:Htop
                    TerraceLength[j] = L_step_2[j] - L_step_2[j+1]
                end
                    
                F = copy(flux_2) # replace the flux with flux_2 for sub step 2
        
                # recalculate the v_array for sub step 2
                v = zeros(Hmax)
                for j in Hbot:Htop-1
                    if L_step_2[j] >= Lmax      
                        if L_step_2[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                        end
            
                    elseif L_step_2[j+1] < Lc
                        v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                    else
                        v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                    end
                end

                #update the L array with sub step 2
                for j in Hbot:Htop
                    if L_step_2[j] >= Lc
                        L_step_2[j+1] += v[j]
                    end
                end
                        
                # find correct Htop after sub step 2
                for j in Hbot:Htop
                    if L_step_2[j] >= Lc
                        if j==Htop 
                            Htop+=1            
                        end
                    end
                end

                F = copy(F_temp) # return the flux to the default
                L_array = copy(L_step_2)    
            end
        end
    
    
        # Both Lc and Lmax correction is needed
        if trigger_Lmax == 1
            if trigger_Lc == 1
            
                 L_step_1 = copy(L_0)  # return to previous step positiion
        
                # return to the old Hot and Hbot
                Htop = copy(Htop_temp)
                Hbot = copy(Hbot_temp)
        
                # Calculate the terrace lengths.
                TerraceLength = zeros(Hmax)
                for j in Hbot:Htop
                    TerraceLength[j] = L_step_1[j] - L_step_1[j+1]
                end
                
                # find length needed to reach exact Lc
                t_length_Lc = Lc - TerraceLength[Htop]
            
                #flux for sub step 1
                flux_1_Lc =  t_length_Lc/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]/2*sticking(TerraceLength[Htop-1])*(1+Ls/(Ls+TerraceLength[Htop-1])))
                # special case for layer 1 and 2
                if Hbot == 1
                    if Htop == 2
                        flux_1_Lc = t_length_Lc/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]*sticking(TerraceLength[Htop-1]))
                    end
                end
            
                # find length needed to reach exact Lmax
                t_length_Lmax = TerraceLength[Hbot]

                #flux for sub step 1
                flux_1_Lmax =  t_length_Lmax/(TerraceLength[Hbot]*sticking(TerraceLength[Hbot]) + TerraceLength[Hbot+1]/2*sticking(TerraceLength[Hbot+1])*(1-Ls/(Ls+TerraceLength[Hbot+1])))
            
                # see which one happens first
                sub_trigger = 0
                if flux_1_Lc >= flux_1_Lmax
                    sub_trigger = 1 # Lmax correction first
                else
                    sub_trigger = 2 # Lc correction first
                end
                        
                if sub_trigger == 1 # Lmax correction first
                    F_temp = copy(F) # save the default flux
                    F = copy(flux_1_Lmax) # replace the flux with flux_1 for sub step 1

        
                    # recalculate the v_array for sub step Lmax
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_1[j] >= Lmax      
                            if L_step_1[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_1[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end

                    
                    #update the L array with sub step Lmax
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lc
                            L_step_1[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub step Lmax
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lc
                            if j==Htop 
                                Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step Lc
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lmax
                            newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    L_step_2 = copy(L_step_1)
                
                    # Calculate the terrace lengths.
                    TerraceLength = zeros(Hmax)
                    for j in Hbot:Htop
                        TerraceLength[j] = L_step_2[j] - L_step_2[j+1]
                    end
                
                    # find length needed to reach exact Lc after Lmax correction
                    t_length_Lc_sub = Lc - TerraceLength[Htop]
            
                    # now find the flux need for Lc correction after Lmax correction
                    flux_1_Lc_sub =  t_length_Lc_sub/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]/2*sticking(TerraceLength[Htop-1])*(1+Ls/(Ls+TerraceLength[Htop-1])))
                    # special case for layer 1 and 2
                    if Hbot == 1
                        if Htop == 2
                            flux_1_Lc_sub = t_length_Lc_sub/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]*sticking(TerraceLength[Htop-1]))
                        end
                    end
                
                    F = copy(flux_1_Lc_sub) # replace the flux with flux_1 for sub step 1

                    # recalculate the v_array for sub step 2
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_2[j] >= Lmax      
                            if L_step_2[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_2[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end
            
                    
                    #update the L array with sub step Lc
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lc
                            L_step_2[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub step Lc
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lc
                            if j==Htop 
                                Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step 3
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lmax
                            newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    L_step_3 = copy(L_step_2)

                
                    # Calculate the terrace lengths.
                    TerraceLength = zeros(Hmax)
                    for j in Hbot:Htop
                        TerraceLength[j] = L_step_3[j] - L_step_3[j+1]
                    end
                    
                    # finish the leftover
                    flux_2 = F_temp - flux_1_Lmax - flux_1_Lc_sub
                
                    F = copy(flux_2)
                
                    # recalculate the v_array for sub step 2
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_3[j] >= Lmax      
                            if L_step_3[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_3[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end
            
                    
                    #update the L array with sub step 3
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lc
                            L_step_3[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub step 3
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lc
                            if j==Htop 
                                 Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step 3
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lmax
                            newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    F = copy(F_temp) # return the flux to the default
                    L_array = copy(L_step_3)
                end
                
                
                if sub_trigger == 2 # Lc correction first
                    F_temp = copy(F) # save the default flux
                    F = copy(flux_1_Lc) # replace the flux with flux_1 for sub step 1

        
                    # recalculate the v_array for sub step Lmax
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_1[j] >= Lmax      
                            if L_step_1[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_1[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end

                    
                    #update the L array with sub step Lc
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lc
                            L_step_1[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub step Lc
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lc
                            if j==Htop 
                                Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step Lmax
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lmax
                            newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    L_step_2 = copy(L_step_1)
                
                    # Calculate the terrace lengths.
                    TerraceLength = zeros(Hmax)
                    for j in Hbot:Htop
                        TerraceLength[j] = L_step_2[j] - L_step_2[j+1]
                    end
                    
                    
                    # find length needed to reach exact Lmax after Lc correction
                    t_length_Lmax_sub = TerraceLength[Hbot]

                    # now find the flux need for Lmax correction after Lc correction
                    flux_1_Lmax_sub =  t_length_Lmax_sub/(TerraceLength[Hbot]*sticking(TerraceLength[Hbot]) + TerraceLength[Hbot+1]/2*sticking(TerraceLength[Hbot+1])*(1-Ls/(Ls+TerraceLength[Hbot+1])))
                
                
                    F = copy(flux_1_Lmax_sub) # replace the flux with flux_1 for sub step 1

                    # recalculate the v_array for sub step 2
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_2[j] >= Lmax      
                            if L_step_2[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_2[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end
            
                    
                    #update the L array with sub step Lmax
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lc
                            L_step_2[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub step Lmax
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lc
                            if j==Htop 
                                Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step 3
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lmax
                            newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    L_step_3 = copy(L_step_2)

                    # Calculate the terrace lengths.
                    TerraceLength = zeros(Hmax)
                    for j in Hbot:Htop
                        TerraceLength[j] = L_step_3[j] - L_step_3[j+1]
                    end
                    
                    # finish the leftover
                    flux_2 = F_temp - flux_1_Lmax_sub - flux_1_Lc
                
                    F = copy(flux_2)
                    # recalculate the v_array for sub step 2
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_3[j] >= Lmax      
                            if L_step_3[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_3[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end
            
                    
                    #update the L array with sub 3
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lc
                            L_step_3[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub 3
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lc
                            if j==Htop 
                                 Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step 3
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lmax
                        newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    F = copy(F_temp) # return the flux to the default
                    L_array = copy(L_step_3)
                end
            end
        end

    end
return L_array, Htop, Hbot

end

function create_profile(layer_lengths_array)
    xvals = []
    yvals = []
    mymaxheight = size(layer_lengths_array)[1]
    for int in 1:mymaxheight
        my_layer_length = layer_lengths_array[int]
        if(my_layer_length == 0)
            break
        end
        xvals = append!(xvals, [my_layer_length])
        yvals = append!(yvals, [int])
    end
    return xvals, yvals
end

function terrace_calc(x)
    terrace = []
    for i in 1:size(x)[1]-1
        terrace = append!(terrace, x[i]-x[i+1])
    end
    return terrace
end

function rms_calc(L_array)
    terrace = zeno.terrace_calc(L_array)
    mean = sum(terrace.*(1:length(terrace)))/sum(terrace)
    rms = sqrt(sum(terrace.*((1-mean:length(terrace)-mean).^2))/sum(terrace))
    return rms, mean
end
end # end module

Main.zeno

In [2]:
using Statistics;using PyPlot; pygui(true)

true

In [3]:
0.005*500000

2500.0

In [6]:
a = 1
Hmax = 2800
F = 0.005
Lc = 0.5
Ls = 0.03
tmax = 500000
Lmax = 3.5
Ld = 0.1
B = 0.00



L_array, Htop, Hbot = zeno.zenogrowth(Lmax, Hmax, tmax; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B)



([3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 2508, 242)

In [12]:



figure()

plot(L_array,1:Hmax,"-o",label = )

xlabel("Width (lattice)")
ylabel("Normalized height")

plt.tight_layout()

In [4]:
a = 1
Hmax = 2800
F = 0.005
Lc = 0.5
Ls = 0.03
tmax = 35000
Lmax = 16
Ld = 0.1
B = 0.00
tmax_array = [50000,100000,150000,200000,250000,300000,350000,400000,450000,500000]
# tmax_array = [50000,100000]

figure()

for i in 1:length(tmax_array)
    
    L_array, Htop, Hbot = zeno.zenogrowth(Lmax, Hmax, tmax_array[i]; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B)
    
    rms, avg_h = zeno.rms_calc(L_array)
    println(rms," ",avg_h)
    
    x,y = zeno.create_profile(L_array)
    x = append!(x,0)
    y = append!(y,length(x))
#     plot(x,y,markersize=6.0,marker="o",label="tmax = $(F*tmax_array[i])" )
    plot(x,y/avg_h,markersize=6.0,marker="o",label="F*t = $(F*tmax_array[i]) layers" )
    



    
end

xlabel("Width (latice)",fontsize=15)
# xlabel("Layer Covrage",fontsize=15)
ylabel("Normalized Height (layers)",fontsize=15)


legend(fontsize=15)

tick_params(labelsize=15)
tight_layout()

4.03107057256545 250.99999999999994
5.21786627957387 501.0000000000001
6.139855294255274 750.9999999999999
6.939816999231031 1001.0000000000001
7.669900560956737 1251.0000000000002
8.354281173873641 1501.0000000000005
9.006685693450878 1751.0
9.635784357352104 2001.0
10.247588015833527 2251.0
10.846205432018222 2501.0


In [4]:
0.005*5000

25.0

In [6]:
a = 1
Hmax = 2800
F = 0.005
Lc = 0.5
Ls = 0.015
tmax = 35000
Lmax = 16
Ld = 0.1
B = 0.00
tmax_array = [1000,2000,3000,4000,5000,6000,8000,14000,16000,20000]
# tmax_array = [50000,100000]

figure()

for i in 1:length(tmax_array)
    
    L_array, Htop, Hbot = zeno.zenogrowth(Lmax, Hmax, tmax_array[i]; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B)
    
    rms, avg_h = zeno.rms_calc(L_array)
    println(rms," ",avg_h)
    
    x,y = zeno.create_profile(L_array)
    x = append!(x,0)
    y = append!(y,length(x))
#     plot(x,y,markersize=6.0,marker="o",label="tmax = $(F*tmax_array[i])" )
    plot(x,y/avg_h,markersize=6.0,marker="o",label="F*t = $(F*tmax_array[i]) layers" )
    



    
end

xlabel("Width (latice)",fontsize=15)
# xlabel("Layer Covrage",fontsize=15)
ylabel("Normalized Height (layers)",fontsize=15)


legend(fontsize=15)

tick_params(labelsize=15)
tight_layout()

1.1238101281082724 6.000000000000001
1.3977173715194842 10.999999999999998
1.57731200673398 15.999999999999996
1.717565190789979 21.0
1.8378417101174513 25.999999999999996
1.9467574570277566 31.0
2.1302367126579056 41.00000000000001
2.5293009192684974 71.00000000000001
2.6386853855931363 81.00000000000001
2.8299281462604964 101.0


In [ ]:
Lmax_array = vcat(1:0.5:2.5,2.6:0.2:3.4,3.5:0.05:4.8,5.0:0.2:6,7:10,10:1:30)
Lmax_array = vcat(1:0.5:2.5,2.6:0.2:3.4,3.5:0.05:5.0,5.2:0.2:7,8:1:30,32:2:50)
Lmax_array = vcat(1:0.5:3,3.2:0.1:3.5,3.55:0.05:5.2,5.4:0.2:8,9:1:30,32:2:50)

# Lmax_array = 10:1:30
println(Lmax_array)

In [ ]:
a = 1
Hmax = 1800
F = 0.005
Lc = 0.5
Ls = 0.015
tmax = 300000
# Lmax = 5
Ld = 0.1
B = 0.00



Lmax_array = vcat(1:1:10)


# figure()
rough = []

for i in 1:length(Lmax_array)
    
    rms, avg_h = zeno.zenogrowth(Lmax_array[i], Hmax, tmax; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B)
    
#     plot(h_f,rms_f,"-o",label="Lmax = $(Lmax_array[i])")
    
    t_layer = (1/F)

    rms_ini = rms[ceil(Int,(tmax - 520*t_layer)):ceil(Int,(tmax - 500*t_layer))]


    rms_final = rms[ceil(Int,(tmax - 20*t_layer)):tmax]


    a = mean(rms_ini)
    b = mean(rms_final)
    c = (b-a)/a
    
#     println(Lmax_array[i])
#     println("mean:",a)
#     println("mean:",b)
#     println("diff:",c*100," %")
    println(c*100)
#     println("_")
    rough = append!(rough,c*100)  
    
end

# plt.plot(1:1500,sqrt.(1:1500),color="black",lw=3)

# tick_params(labelsize=15)
# xlabel("Thickness (layers)",fontsize=15)
# ylabel("RMS roughness (layers)",fontsize=15)


# legend(fontsize=15)

# tick_params(labelsize=15)
# tight_layout()

0.0
0.0
7.342494149583839e-12
0.03898565704253541
2.624520826804151
7.980623801932432
11.114309632168151


In [ ]:
pygui(false)

figure()

plot(Lmax_array,rough,"-o")

tight_layout()

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x00000279BAAD20E0>

In [ ]:
1/8/Ls+0.5

In [ ]:
sqrt(1/Ls)